In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
import random

In [5]:
def channel_scraper(url, numvideos):
    data = []
    cols = ["Channel","Subscribers", "Video Name", "Views", "Link"]

    #mutes audio
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--mute-audio")
    try:
        #sets up driver and goes into video tab
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        #video_button = driver.find_element_by_xpath('//*[@id="tabsContent"]/paper-tab[2]/div')
        video_button = WebDriverWait(driver,7).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="tabsContent"]/paper-tab[2]/div')))
        video_button.click()
        time.sleep(3)

        #data collections
        channel = driver.find_element_by_xpath('//*[@id="text-container"]').text
        subs = driver.find_element_by_xpath('//*[@id="subscriber-count"]').text
        videos = driver.find_elements_by_class_name("yt-simple-endpoint style-scope ytd-grid-video-renderer")
        vidnames = driver.find_elements_by_xpath('//*[@id="video-title"]')
        views = driver.find_elements_by_xpath('//*[@id="metadata-line"]/span[1]')
        thumbnails = driver.find_elements_by_id("video-title")
        links = [thumb.get_attribute('href') for thumb in thumbnails]
    except:
        driver.close()
        return None
    
    if len(vidnames) == 0:
        driver.close()
        return None

    #gathers the top 100 most recent videos
    n = 100000
    counter = 1
    while len(vidnames) < numvideos:
        script = "window.scrollTo(0," + str(n) +")"
        driver.execute_script(script) 
        n *= 2
        #timer.sleep(2)
        vidnames = driver.find_elements_by_xpath('//*[@id="video-title"]')
        counter += 1
        if counter >= 100:
            break
    #ensures data is uniform
    while(not (len(vidnames) == len(views) == len(links))):
        vidnames = driver.find_elements_by_xpath('//*[@id="video-title"]')
        views = driver.find_elements_by_xpath('//*[@id="metadata-line"]/span[1]')
        thumbnails = driver.find_elements_by_id("video-title")
        links = [thumb.get_attribute('href') for thumb in thumbnails]
    
    #reformating data
    #subscriber count to int
    if 'M subscribers' in subs:
        subs = int(float(subs.replace("M subscribers",""))*1000000)
    elif 'K subscribers' in subs:
        subs = int(float(subs.replace("K subscribers",""))*1000)
    
    views = [view.text for view in views]
    for i in range(len(views)):
        if 'B views' in views[i]:
            views[i] = int(float(views[i].replace('B views','')) * 1000000000)
        elif 'M views' in views[i]:
            views[i] = int(float(views[i].replace('M views','')) * 1000000)
        elif 'K views' in views[i]:
            views[i] = int(float(views[i].replace('K views','')) * 1000)
        else:
            views[i] = 0
    
    #formats into dataframe
    if len(vidnames) == len(views) == len(links):
        for i in range(len(vidnames)):
            #if views[i] >= 1000:
            data.append((channel, subs, vidnames[i].text, views[i], links[i]))
    #returns resulting data frame
    driver.close()
    result = pd.DataFrame(data, columns = cols)
    return result


In [6]:
def rand_walk(seed, steps):
    #mutes audio
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--mute-audio")

    driver = webdriver.Chrome(options=chrome_options)
    result = []
    for i in range(steps+1):
        driver.get(seed)
        driver.maximize_window()
        time.sleep(5)    

        #Gathers the future links
        thumbnails = driver.find_elements_by_id("thumbnail")
        links = [thumb.get_attribute("href") for thumb in thumbnails]
        links.pop()
        
        #gathers the current video
        curr_vid = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
        curr_channel = driver.find_element_by_xpath('//*[@id="text"]/a').text

        
        
        #checks if link is empty
        #if it is, we keep the same link and retry
        try:
            if not links == None and not len(links) == 0:
                #appends the results if the next link is valid

                #makes sure link is valid
                #randomly chooses a video from the top 10 videos
                if len(links) >= 9:
                    choice = random.choice(range(0,9))
                else:
                    choice = random.choice(range(0,len(links)-1))
                newlink = links[choice]

                if newlink == None:
                    i = i -1
                else:
                    while not 'https://www.youtube.com/watch' == newlink[0:29]:
                        print('https://www.youtube.com/watch' == newlink[0:29])
                        if len(links) >= 9:
                            choice = random.choice(range(0,9))
                        else:
                            choice = random.choice(range(0,len(links)-1))
                        newlink = links[choice]
                        if newlink == None:
                            i = i -1
                            break

                    result.append(curr_channel)
                    result.append(curr_vid)
                    result.append(len(links))
                    seed = newlink
            else:
                i = i - 1
        except:
            i = i - 1


        
    driver.quit()
    return result

In [7]:
def vidsim(startvid,steps, num_sims):
    #sets up appropriate amounts of columns
    cols = ["Origin Channel","Origin Video", "recommendation #"]
    data = []
    for i in range(steps):
        cols.append("Step: " + str(i+1) + " Channel")
        cols.append("Step: " + str(i+1) + " Video")
        cols.append("vid " + str(i+1) + " recommendations #")
   
    #gets the data from the video scraper
    for i in range(num_sims):
        scrape = rand_walk(startvid,steps)
        scrape = tuple(scrape)
        data.append(scrape)
    #imports data into df
    df1 = pd.DataFrame(data, columns = cols)
    #return data
    return df1
    
    

In [21]:
vidlist= ['https://www.youtube.com/watch?v=V6Q_nlSULio']
num_sim = 5
steps = 5
df = pd.DataFrame()
for vid in vidlist:
    df = df.append(vidsim(vid,steps,num_sim), ignore_index = True)

In [8]:
#Takes in a list of links and create a markov matrix from their recommended channels
def vidcounter(Media, links):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--mute-audio")
    driver = webdriver.Chrome(options = chrome_options)
    
    result = pd.DataFrame()
    
    for url in links:
        try:
            prob = {}
            driver.get(url)
            driver.maximize_window()
            time.sleep(3)
            driver.execute_script("window.scrollTo(0,1000)")

            channels = driver.find_elements_by_id('channel-name')
            currmedia = Media
            currchannel = driver.find_element_by_xpath('//*[@id="text"]/a').text
            currvideo = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
            currviews = driver.find_element_by_xpath('//*[@id="count"]/yt-view-count-renderer/span[1]').text

            channels = channels[0:10]
            for channel in channels:
                if not channel.text in prob:
                    prob[channel.text] = 1
                else:
                    prob[channel.text] = prob[channel.text] + 1

            df = pd.DataFrame(prob.items(), columns = ["Next Channel","Count"])
            currmedia = currmedia
            currchannel = currchannel
            currvideo = currvideo
            currviews = currviews.strip(" views").replace(",","")
            df["Category"] = currmedia
            df["Current Channel"] = currchannel
            df["Current Video"] = currvideo
            df["Current Views"] = currviews
            df = df[["Category","Current Channel","Current Video","Current Views","Next Channel","Count"]].head(10)
            result = result.append(df,ignore_index = True)
        except:
            print("Nothing is gathered")
    driver.close()
    return result

In [11]:
#gets the distribution of watching a domain of video of going to the next video
#and creates csv files for each category
def probcsv(category, minchannel,minvidchannel, vidsamplesize):
    #finds all channels with a specific tag and takes a random sample of 10 channels
    tags = pd.read_csv("Hard Tagged Channels.csv")
    news = tags[tags["TAGS"].str.contains(category)]["CHANNEL_ID"].tolist()
    news = ['https://www.youtube.com/channel/' + user for user in news]

    #how many youtube channels are sampled
    #either the minvidchannel or the maximum amount of available channels is used
    try:
        data = random.sample(news,minchannel) #this is used
    except:
        data = news
        
    channelvids = pd.DataFrame()

    #and gets the sampling distribution

    for channel in data:
        channelvids = channelvids.append(channel_scraper(channel,minvidchannel))
    channelvids[channelvids["Views"] >= 10000]
    try:
        samplingvids = channelvids["Link"].sample(n=vidsamplesize).tolist()
    except:
        samplingvids = channelvids["Link"].tolist()
        
    probability = vidcounter(category, samplingvids)
    probability.to_csv(category + " probability.csv")

In [12]:
uniquetags = [ 'Educational']
for tag in uniquetags:
    #using these tags, take a min of 100 videos if possible, and simulate with 30 videos
    #takes in number of channels, vids from each channel, number of vids sampled
    probcsv(tag,100,300,100)